In [9]:
def filter_attribute(X):
    continous_cols = []
    nominal_cols = []
    for i in range(X.shape[1]):
        col = X[:, i]
        
        not_cont = True
        for ele in col:
            if ele == 0 or ele == 1:
                pass
            else:
                not_cont = False

        if not_cont:
            nominal_cols.append(col)
        else:
            continous_cols.append(col)
            
    return continous_cols, nominal_cols

# Information theoretic

In [10]:
from os import listdir

import numpy as np
import pandas as pd

In [11]:
def entropy(a):
    probs = [np.mean(a == c) for c in set(a)]
    return np.sum(-p * np.log2(p) for p in probs if p > 0)

def joint_entropy_2val(a, b):
    probs = []
    for c1 in set(a):
        for c2 in set(b):
            probs.append(np.mean(np.logical_and(a == c1, b == c2)))

    return np.sum(-p * np.log2(p) for p in probs if p > 0)

def infoMetas(dataset):
    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    cont, nom = filter_attribute(X)
    
    #Convert continous columns into frequency bins
    for attr in cont:
        attr_vals = pd.cut(attr, 10, labels=list(range(10)), retbins=True)[0]
        nom.append(attr_vals)
    
    class_ent = entropy(y)
    
    attr_entr = np.zeros(len(nom))
    attr_entr_norm = np.zeros(attr_entr.shape)
    mut_info = np.zeros(attr_entr.shape)
    joint_entr = np.zeros(attr_entr.shape)
    
    for i in range(len(nom)):
        attr  = nom[i]
        attr_entr_cur = entropy(attr)
        attr_entr[i] = attr_entr_cur
        attr_entr_norm[i] = attr_entr_cur / np.log2(len(attr))
        
        joint_entr_cur = joint_entropy_2val(attr, y)
        joint_entr[i] = joint_entr_cur
        
        mut_info[i] = class_ent + attr_entr_cur - joint_entr_cur
    
    '''ClassEnt, AttrEnt[Min, Mean, Max], JointEnt, MutInfo[Min, Mean, Max], EquiAttr, NoiseRatio'''
    metas_list = [class_ent, attr_entr_norm.min(), attr_entr_norm.mean(), attr_entr_norm.max(), joint_entr.mean(), 
                  mut_info.min(), mut_info.mean(), mut_info.max(), class_ent / mut_info.mean(), 
                  np.abs(attr_entr.mean() - mut_info.mean()) / mut_info.mean()]
    return metas_list

# Statistical

In [12]:
from scipy.stats import skew
from scipy.stats import kurtosis

In [13]:
def statMetas(dataset):
    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    #Include all columns as continous attributes
    cont = X
        
    std_x = np.zeros(len(cont))
    skew_x = np.zeros(std_x.shape)
    kurtosis_x = np.zeros(std_x.shape)

    for i in range(len(cont)):
        attr = cont[i]
        std_x[i] = attr.std()
        skew_x[i] = skew(attr)
        kurtosis_x[i] = kurtosis(attr)
    
    '''StandardDev[Min, Mean, Max], Skewness[Min, Mean, Max], Kurtosis[Min, Mean, Max]'''
    metas_list = [std_x.min(), std_x.mean(), std_x.max(), skew_x.min(), skew_x.mean(), skew_x.max(), 
                  kurtosis_x.min(), kurtosis_x.mean(), kurtosis_x.max()]
    return metas_list

# Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
def decisionTreeMetas(dataset):
    X = dataset[:, :-1]
    Y = dataset[:, -1]
    
    clf = DecisionTreeClassifier()
    clf.fit(X, Y)

    model = clf.tree_
    n_nodes = model.node_count
    children_left = model.children_left
    children_right = model.children_right
    feature = model.feature
    threshold = model.threshold

    # The tree structure can be traversed to compute various properties such
    # as the depth of each node and whether or not it is a leaf.
    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    n_leaves = 0
    stack = [(0, -1)]  # seed is the root node id and its parent depth
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1

        # If we have a test node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True
            n_leaves += 1

    #Tree Width
    num_left = 0
    cur = 0
    while cur != -1:
        cur = model.children_left[cur]
        num_left += 1

    num_right = 0
    cur = 0
    while cur != -1:
        cur = model.children_right[cur]
        num_right += 1

    n_width = num_left + num_right - 1

    #Branches
    branch_length = []
    for i in range(n_nodes):
        if is_leaves[i]:
            branch_length.append(node_depth[i])
    branch_length = np.array(branch_length)

    #Nodes per level
    nodes_level = np.zeros(shape=model.max_depth)
    for i in node_depth[1:]:
        nodes_level[i-1] += 1

    #Attribute occurence
    feature_occr = np.zeros(shape=clf.n_features_)
    for i in feature:
        if not i == -2:
            feature_occr[i] += 1

    #Output
    '''treewidth, treeheight, NoNode, NoLeave, maxLevel, meanLevel, devLevel, 
       ShortBranch, meanBranch, devBranch, maxAtt, minAtt, 
       meanAtt, devAtt'''
    metas_list = [n_width, model.max_depth, n_nodes, n_leaves, max(nodes_level), nodes_level.mean(), nodes_level.std(),
                  min(branch_length), branch_length.mean(), branch_length.std(), max(feature_occr), min(feature_occr), 
                  feature_occr.mean(), feature_occr.std()]
    return metas_list

# Calculating and saving

In [30]:
datasets = listdir("G:\\main_folder")

for filename in datasets:
    dir_name = "G:\\main_folder\\" + filename
    
    dataset = pd.read_csv(dir_name, header = None)
    dataset = dataset.as_matrix()
    print('Calculating for: {}'.format(filename))
    
    try:
        temp = infoMetas(dataset) + statMetas(dataset) + decisionTreeMetas(dataset)
    except ValueError:
        print('Dataset without continous attributes found: ', filename)
    else:
        has_nan = np.any(np.isnan(temp))
        has_inf = np.any(np.isinf(temp))
        if has_nan and has_inf:
            print('Dataset has both nan and inf: ', filename)
            break
        elif has_nan:
            print('Dataset has nan: ', filename)
            break
        elif has_inf:
            print('Dataset has inf: ', filename)
            break
    
    temp = np.array(temp)
    temp.dump('G:\\Classical-Decision_Tree\\' + filename + '.npy')

c:\users\lenovo\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
c:\users\lenovo\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\lenovo\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  # This is added back by InteractiveShellApp.init_path()


Calculating for: 0.txt
Calculating for: 1.txt
Calculating for: 10.txt
Calculating for: 100.txt
Calculating for: 1000.txt
Calculating for: 10000.txt
Calculating for: 10001.txt
Calculating for: 10002.txt
Calculating for: 10003.txt
Calculating for: 10004.txt
Calculating for: 10005.txt
Calculating for: 10006.txt
Calculating for: 10007.txt
Calculating for: 10008.txt
Calculating for: 10009.txt
Calculating for: 1001.txt
Calculating for: 10010.txt
Calculating for: 10011.txt
Calculating for: 10012.txt
Calculating for: 10013.txt
Calculating for: 10014.txt
Calculating for: 10015.txt
Calculating for: 10016.txt
Calculating for: 10017.txt
Calculating for: 10018.txt
Calculating for: 10019.txt
Calculating for: 1002.txt
Calculating for: 10020.txt
Calculating for: 10021.txt
Calculating for: 10022.txt
Calculating for: 10023.txt
Calculating for: 10024.txt
Calculating for: 10025.txt
Calculating for: 10026.txt
Calculating for: 10027.txt
Calculating for: 10028.txt
Calculating for: 10029.txt
Calculating for: 

Calculating for: 10272.txt
Calculating for: 10273.txt
Calculating for: 10274.txt
Calculating for: 10275.txt
Calculating for: 10276.txt
Calculating for: 10277.txt
Calculating for: 10278.txt
Calculating for: 10279.txt
Calculating for: 1028.txt
Calculating for: 10280.txt
Calculating for: 10281.txt
Calculating for: 10282.txt
Calculating for: 10283.txt
Calculating for: 10284.txt
Calculating for: 10285.txt
Calculating for: 10286.txt
Calculating for: 10287.txt
Calculating for: 10288.txt
Calculating for: 10289.txt
Calculating for: 1029.txt
Calculating for: 10290.txt
Calculating for: 10291.txt
Calculating for: 10292.txt
Calculating for: 10293.txt
Calculating for: 10294.txt
Calculating for: 10295.txt
Calculating for: 10296.txt
Calculating for: 10297.txt
Calculating for: 10298.txt
Calculating for: 10299.txt
Calculating for: 103.txt
Calculating for: 1030.txt
Calculating for: 10300.txt
Calculating for: 10301.txt
Calculating for: 10302.txt
Calculating for: 10303.txt
Calculating for: 10304.txt
Calcul

Calculating for: 10547.txt
Calculating for: 10548.txt
Calculating for: 10549.txt
Calculating for: 1055.txt
Calculating for: 10550.txt
Calculating for: 10551.txt
Calculating for: 10552.txt
Calculating for: 10553.txt
Calculating for: 10554.txt
Calculating for: 10555.txt
Calculating for: 10556.txt
Calculating for: 10557.txt
Calculating for: 10558.txt
Calculating for: 10559.txt
Calculating for: 1056.txt
Calculating for: 10560.txt
Calculating for: 10561.txt
Calculating for: 10562.txt
Calculating for: 10563.txt
Calculating for: 10564.txt
Calculating for: 10565.txt
Calculating for: 10566.txt
Calculating for: 10567.txt
Calculating for: 10568.txt
Calculating for: 10569.txt
Calculating for: 1057.txt
Calculating for: 10570.txt
Calculating for: 10571.txt
Calculating for: 10572.txt
Calculating for: 10573.txt
Calculating for: 10574.txt
Calculating for: 10575.txt
Calculating for: 10576.txt
Calculating for: 10577.txt
Calculating for: 10578.txt
Calculating for: 10579.txt
Calculating for: 1058.txt
Calcu

Calculating for: 10821.txt
Calculating for: 10822.txt
Calculating for: 10823.txt
Calculating for: 10824.txt
Calculating for: 10825.txt
Calculating for: 10826.txt
Calculating for: 10827.txt
Calculating for: 10828.txt
Calculating for: 10829.txt
Calculating for: 1083.txt
Calculating for: 10830.txt
Calculating for: 10831.txt
Calculating for: 10832.txt
Calculating for: 10833.txt
Calculating for: 10834.txt
Calculating for: 10835.txt
Calculating for: 10836.txt
Calculating for: 10837.txt
Calculating for: 10838.txt
Calculating for: 10839.txt
Calculating for: 1084.txt
Calculating for: 10840.txt
Calculating for: 10841.txt
Calculating for: 10842.txt
Calculating for: 10843.txt
Calculating for: 10844.txt
Calculating for: 10845.txt
Calculating for: 10846.txt
Calculating for: 10847.txt
Calculating for: 10848.txt
Calculating for: 10849.txt
Calculating for: 1085.txt
Calculating for: 10850.txt
Calculating for: 10851.txt
Calculating for: 10852.txt
Calculating for: 10853.txt
Calculating for: 10854.txt
Calc

...

Calculating for: 8795.txt
Calculating for: 8796.txt
Calculating for: 8797.txt
Calculating for: 8798.txt
Calculating for: 8799.txt
Calculating for: 88.txt
Calculating for: 880.txt
Calculating for: 8800.txt
Calculating for: 8801.txt
Calculating for: 8802.txt
Calculating for: 8803.txt
Calculating for: 8804.txt
Calculating for: 8805.txt
Calculating for: 8806.txt
Calculating for: 8807.txt
Calculating for: 8808.txt
Calculating for: 8809.txt
Calculating for: 881.txt
Calculating for: 8810.txt
Calculating for: 8811.txt
Calculating for: 8812.txt
Calculating for: 8813.txt
Calculating for: 8814.txt
Calculating for: 8815.txt
Calculating for: 8816.txt
Calculating for: 8817.txt
Calculating for: 8818.txt
Calculating for: 8819.txt
Calculating for: 882.txt
Calculating for: 8820.txt
Calculating for: 8821.txt
Calculating for: 8822.txt
Calculating for: 8823.txt
Calculating for: 8824.txt
Calculating for: 8825.txt
Calculating for: 8826.txt
Calculating for: 8827.txt
Calculating for: 8828.txt
Calculating for: 

Calculating for: 908.txt
Calculating for: 9080.txt
Calculating for: 9081.txt
Calculating for: 9082.txt
Calculating for: 9083.txt
Calculating for: 9084.txt
Calculating for: 9085.txt
Calculating for: 9086.txt
Calculating for: 9087.txt
Calculating for: 9088.txt
Calculating for: 9089.txt
Calculating for: 909.txt
Calculating for: 9090.txt
Calculating for: 9091.txt
Calculating for: 9092.txt
Calculating for: 9093.txt
Calculating for: 9094.txt
Calculating for: 9095.txt
Calculating for: 9096.txt
Calculating for: 9097.txt
Calculating for: 9098.txt
Calculating for: 9099.txt
Calculating for: 91.txt
Calculating for: 910.txt
Calculating for: 9100.txt
Calculating for: 9101.txt
Calculating for: 9102.txt
Calculating for: 9103.txt
Calculating for: 9104.txt
Calculating for: 9105.txt
Calculating for: 9106.txt
Calculating for: 9107.txt
Calculating for: 9108.txt
Calculating for: 9109.txt
Calculating for: 911.txt
Calculating for: 9110.txt
Calculating for: 9111.txt
Calculating for: 9112.txt
Calculating for: 9

Calculating for: 9365.txt
Calculating for: 9366.txt
Calculating for: 9367.txt
Calculating for: 9368.txt
Calculating for: 9369.txt
Calculating for: 937.txt
Calculating for: 9370.txt
Calculating for: 9371.txt
Calculating for: 9372.txt
Calculating for: 9373.txt
Calculating for: 9374.txt
Calculating for: 9375.txt
Calculating for: 9376.txt
Calculating for: 9377.txt
Calculating for: 9378.txt
Calculating for: 9379.txt
Calculating for: 938.txt
Calculating for: 9380.txt
Calculating for: 9381.txt
Calculating for: 9382.txt
Calculating for: 9383.txt
Calculating for: 9384.txt
Calculating for: 9385.txt
Calculating for: 9386.txt
Calculating for: 9387.txt
Calculating for: 9388.txt
Calculating for: 9389.txt
Calculating for: 939.txt
Calculating for: 9390.txt
Calculating for: 9391.txt
Calculating for: 9392.txt
Calculating for: 9393.txt
Calculating for: 9394.txt
Calculating for: 9395.txt
Calculating for: 9396.txt
Calculating for: 9397.txt
Calculating for: 9398.txt
Calculating for: 9399.txt
Calculating for

Calculating for: 9650.txt
Calculating for: 9651.txt
Calculating for: 9652.txt
Calculating for: 9653.txt
Calculating for: 9654.txt
Calculating for: 9655.txt
Calculating for: 9656.txt
Calculating for: 9657.txt
Calculating for: 9658.txt
Calculating for: 9659.txt
Calculating for: 966.txt
Calculating for: 9660.txt
Calculating for: 9661.txt
Calculating for: 9662.txt
Calculating for: 9663.txt
Calculating for: 9664.txt
Calculating for: 9665.txt
Calculating for: 9666.txt
Calculating for: 9667.txt
Calculating for: 9668.txt
Calculating for: 9669.txt
Calculating for: 967.txt
Calculating for: 9670.txt
Calculating for: 9671.txt
Calculating for: 9672.txt
Calculating for: 9673.txt
Calculating for: 9674.txt
Calculating for: 9675.txt
Calculating for: 9676.txt
Calculating for: 9677.txt
Calculating for: 9678.txt
Calculating for: 9679.txt
Calculating for: 968.txt
Calculating for: 9680.txt
Calculating for: 9681.txt
Calculating for: 9682.txt
Calculating for: 9683.txt
Calculating for: 9684.txt
Calculating for

Calculating for: 9936.txt
Calculating for: 9937.txt
Calculating for: 9938.txt
Calculating for: 9939.txt
Calculating for: 994.txt
Calculating for: 9940.txt
Calculating for: 9941.txt
Calculating for: 9942.txt
Calculating for: 9943.txt
Calculating for: 9944.txt
Calculating for: 9945.txt
Calculating for: 9946.txt
Calculating for: 9947.txt
Calculating for: 9948.txt
Calculating for: 9949.txt
Calculating for: 995.txt
Calculating for: 9950.txt
Calculating for: 9951.txt
Calculating for: 9952.txt
Calculating for: 9953.txt
Calculating for: 9954.txt
Calculating for: 9955.txt
Calculating for: 9956.txt
Calculating for: 9957.txt
Calculating for: 9958.txt
Calculating for: 9959.txt
Calculating for: 996.txt
Calculating for: 9960.txt
Calculating for: 9961.txt
Calculating for: 9962.txt
Calculating for: 9963.txt
Calculating for: 9964.txt
Calculating for: 9965.txt
Calculating for: 9966.txt
Calculating for: 9967.txt
Calculating for: 9968.txt
Calculating for: 9969.txt
Calculating for: 997.txt
Calculating for:

In [27]:
import numpy as np

names_list = 'ClassEnt, AttrEntMin, AttrEntMean, AttrEntMax, JointEnt, MutInfoMin, MutInfoMean, MutInfoMax, EquiAttr,' + \
             'NoiseRatio, StandardDevMin, StandardDevMean, StandardDevMax, SkewnessMin, SkewnessMean, SkewnessMax,' + \
             'KurtosisMin, KurtosisMean, KurtosisMax, treewidth, treeheight, NoNode, NoLeave, maxLevel, meanLevel, devLevel,' + \
             'ShortBranch, meanBranch, devBranch, maxAtt, minAtt, meanAtt, devAtt'
names_list = np.array([x.strip() for x in names_list.split(',')])

In [28]:
names_list.dump('meta_features_names.npy')